In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split


In [2]:
spark = SparkSession \
    .builder \
    .appName("StructuredNetworkWordCount") \
    .enableHiveSupport() \
    .config("spark.sql.warehouse.dir","/user/hive/warehouse") \
    .getOrCreate()
	# Create DataFrame representing the stream of input lines from connection to localhost:9999
spark.sparkContext.setLogLevel("ERROR")

2021-09-14 06:50:35,290 WARN util.Utils: Your hostname, spark-VirtualBox resolves to a loopback address: 127.0.1.1; using 192.168.29.124 instead (on interface ens33)
2021-09-14 06:50:35,298 WARN util.Utils: Set SPARK_LOCAL_IP if you need to bind to another address
2021-09-14 06:50:37,944 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2021-09-14 06:50:44,319 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
2021-09-14 06:50:44,320 WARN util.Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
2021-09-14 06:50:44,321 WARN util.Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [3]:
lines = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()

In [4]:
words = lines.select(
   explode(
       split(lines.value, " ")
   ).alias("word")
)


In [5]:
wordCounts = words.groupBy("word").count()

def func(batch_df, batch_id):
	print('Writing to hive {}'.format(batch_id))
	batch_df.write.mode("append").saveAsTable("WordCount_spark1")
    

In [ ]:
query = wordCounts.writeStream.foreachBatch(func).outputMode("update").start()
query.awaitTermination()

Writing to hive 0


Writing to hive 1
